In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')

In [ ]:
df.head()

In [ ]:
#checking for the balace of classes
clas=[0,0,0,0,0]
for p in df['diagnosis']:
  clas[p] = clas[p]+1

In [ ]:
import matplotlib.pyplot as plt
plt.plot(clas,'ro')

In [ ]:
def clahe_greyscale(input_path):
    img = cv2.imread(input_path)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl1 = clahe.apply(gray_image)   
    #cl1=cv2.resize(cl1,(512,512))
    return cl1
def equalize_hist(input_path):
    img = cv2.imread(input_path)
    for c in range(0, 2):
        img[:,:,c] = cv2.equalizeHist(img[:,:,c])
    #img=cv2.resize(img,(512,512))
    return img

def clahe_rgb(bgr):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    lab_planes = cv2.split(lab)
    gridsize = 5
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(gridsize,gridsize))
    lab_planes[0] = clahe.apply(lab_planes[0])
    lab = cv2.merge(lab_planes)
    bgr2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    #bgr2=cv2.resize(bgr2,(512,512))
    return bgr2

In [ ]:
from imblearn.over_sampling import RandomOverSampler 
import numpy as np
ros =RandomOverSampler(random_state=0)
X1=df['id_code']
X1=np.array(X1).reshape(-1,1)
Y1=df['diagnosis']
X_r,Y_r=ros.fit_resample(X1,Y1)
from collections import Counter
print(sorted(Counter(Y_r).items()))

In [ ]:
def scaleRadius(img,scale):
  x=img[img.shape[0]//2,:,:].sum(1)
  r=(x>x.mean()/10).sum()/2
  s=scale*1.0/r
  return cv2.resize(img,(0,0),fx=s,fy=s)

In [ ]:
dfs = pd.concat([pd.DataFrame(X_r),pd.DataFrame(Y_r)],axis=1)
dfs.columns=['id_code','diagnosis']

In [ ]:
dfs[dfs['id_code']=='0024cdab0c1e']

In [ ]:
def preP(path):
  img = cv2.imread(path)
  
  hsv1=cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
  h,s,v=cv2.split(hsv1)
  hsv1=cv2.merge((h,s,v))
  img2=cv2.cvtColor(hsv1,cv2.COLOR_HSV2RGB)

  img_yuv = cv2.cvtColor(img2, cv2.COLOR_BGR2YUV)

  img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])


  img2 = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
  img2 = cv2.GaussianBlur(img2,(5,5),cv2.BORDER_DEFAULT)
  

  return img2

In [ ]:
import numpy as np
import skimage as sk
import random
from tqdm import tqdm
import os
import cv2
from scipy.ndimage import rotate
from skimage.io import imshow, show

def crop_image(img,tol=7):
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

j=0
from tensorflow.keras.preprocessing.image import img_to_array
tr = []
i=0
for i,image in enumerate(tqdm(os.listdir('../input/aptos2019-blindness-detection/train_images'))):
    x=dfs[dfs['id_code']==image.split('.')[0]].values
    for row in x:
      scale = 300
      img = preP('../input/aptos2019-blindness-detection/train_images/'+image)
      img = scaleRadius(img,scale)
      
      h,w,_ = img.shape
      img1=cv2.resize(crop_image(img[:,:,0]),(w,h))
      img2=cv2.resize(crop_image(img[:,:,1]),(w,h))
      img3=cv2.resize(crop_image(img[:,:,2]),(w,h))
      img[:,:,0]=img1
      img[:,:,1]=img2
      img[:,:,2]=img3
      imgr = cv2.resize(img,(224,224))
      Y= row[1]
      tr.append([imgr,Y])
        

In [ ]:
from sklearn.utils import shuffle
tr =shuffle(tr)

In [ ]:
X=[]
Y=[]
for image,label in tr:
    X.append(image)
    Y.append(label)
X=np.array(X).reshape(-1,224,224,3)
X.shape

In [ ]:
Y = pd.get_dummies(Y).values
Y_t = np.empty(Y.shape, dtype=Y.dtype)
Y_t[:, 4] = Y[:, 4]

for i in range(3, -1, -1):
    Y_t[:, i] = np.logical_or(Y[:, i], Y_t[:, i+1])


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    X,Y_t, 
    test_size=0.15, 
    random_state=1009
)
X=[]
Y=[]
Y_t=[]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 32
def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  
        fill_mode='constant',
        cval=0.,  
        horizontal_flip=True,  
        vertical_flip=True,  
    )
  
data_generator = create_datagen().flow(x_train, y_train, batch_size=BATCH_SIZE)

In [ ]:
from keras.applications import DenseNet121
densenet = DenseNet121(
    weights='../input/petfinder-external/DenseNet-BC-121-32-no-top.h5',
    include_top=False,
    input_shape=(224,224,3)
)

In [ ]:
from keras.models import Sequential
from keras import layers
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam


model = Sequential()
model.add(densenet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='sigmoid'))
model.summary()

model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.00005),metrics=['accuracy'])


In [ ]:
model.fit_generator(
    data_generator,
    steps_per_epoch=x_train.shape[0] / BATCH_SIZE,
    epochs=15,
    validation_data=(x_val, y_val) 
)

In [ ]:
dt = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
dt.head()

In [ ]:
tr = []
ID=[]
i=0
for image in os.listdir('../input/aptos2019-blindness-detection/test_images'):
    id_ = image.split('.')[0]
    scale = 300
    img = preP('../input/aptos2019-blindness-detection/test_images/'+image)
   
    img = scaleRadius(img,scale)
    h,w,_ = img.shape
    img1=cv2.resize(crop_image(img[:,:,0]),(w,h))
    img2=cv2.resize(crop_image(img[:,:,1]),(w,h))
    img3=cv2.resize(crop_image(img[:,:,2]),(w,h))
    img[:,:,0]=img1
    img[:,:,1]=img2
    img[:,:,2]=img3
    imgr = cv2.resize(img,(224,224))
    tr.append([id_,imgr])

In [ ]:
T=[]
ID=[]
for id_,image in tr:
    T.append(image)
    ID.append(id_)

In [ ]:
T=np.array(T).reshape(-1,224,224,3)

In [ ]:
Y1=model.predict(T) >0.5

In [ ]:

Y1 = Y1.astype(int).sum(axis=1) - 1


In [ ]:
sub = pd.concat([pd.DataFrame(ID),pd.DataFrame(Y1)],axis=1)

In [ ]:
sub.columns=['id_code','diagnosis']

In [ ]:
sub.head(10)

In [ ]:
sub.to_csv('submission.csv',index=False)